# Задание 2. SQL

### Задание 2.1. Очень усердные ученики. Код для решения задачи:

SELECT COUNT(1) AS count_of_diligent_students
FROM (
    SELECT st_id
    FROM peas
    WHERE correct = true
    GROUP BY st_id
    HAVING COUNT(1) >= 20
) AS diligent_students

### Задание 2.2. Оптимизация воронки. Код для решения задачи:


--активные пользователи
WITH 
active_users AS (
    SELECT st_id
    FROM peas
    WHERE correct = TRUE
    GROUP BY st_id
    HAVING COUNT(*) > 10
),

--активные пользователи по математике
active_math AS (
    SELECT st_id
    FROM peas
    WHERE correct = TRUE AND subject = 'Math'
    GROUP BY st_id
    HAVING COUNT(*) >= 2
),

--покупки
purchases AS (
    SELECT st_id,
           SUM(money) AS total_money,
           BOOL_OR(subject = 'Math') AS bought_math
    FROM final_project_check
    GROUP BY st_id
)


--основной запрос
SELECT 
    s.test_grp,

    --ARPU
    COALESCE(SUM(p.total_money), 0)::float / COUNT(DISTINCT s.st_id) AS arpu,

    --ARPAU   
    COALESCE(SUM(p.total_money), 0)::float 
        / NULLIF(COUNT(DISTINCT CASE WHEN a.st_id IS NOT NULL THEN s.st_id END), 0) AS arpau,
    
    --CR в покупку
    COUNT(DISTINCT CASE WHEN p.st_id IS NOT NULL THEN s.st_id END)::float 
        / COUNT(DISTINCT s.st_id) AS cr,

    --CR активного пользователя в покупку
    COUNT(DISTINCT CASE WHEN a.st_id IS NOT NULL AND p.st_id IS NOT NULL THEN s.st_id END)::float
        / NULLIF(COUNT(DISTINCT CASE WHEN a.st_id IS NOT NULL THEN s.st_id END), 0) AS cr_active,

    --CR пользователя из активности по математике 
    COUNT(DISTINCT CASE WHEN am.st_id IS NOT NULL AND p.bought_math THEN s.st_id END)::float
        / NULLIF(COUNT(DISTINCT am.st_id), 0) AS cr_math_active

FROM studs s
LEFT JOIN purchases p ON s.st_id = p.st_id
LEFT JOIN active_users a ON s.st_id = a.st_id
LEFT JOIN active_math am ON s.st_id = am.st_id
GROUP BY s.test_grp